In [1]:
import bs4
import pandas as pd
import requests
import cx_Oracle
import pymysql
from tqdm import tqdm
import project_utils as mu 
import time
import random 
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc
font_path = 'C:/Windows/Fonts/gulim.ttc'
font = font_manager.FontProperties(fname=font_path).get_name()
rc('font',family = font)
import numpy as np
import seaborn as sns
import plotly
import plotly.graph_objs as go
plotly.offline.init_notebook_mode(connected = True)
import plotly.express as px

In [2]:
mu.db_open()
my_df = mu.oracle_execute('select * from LOL_MP')
mu.oracle_close()

oracle open!


C:\Users\user\Anaconda3\lib\site-packages\pandas\io\sql.py:762: UserWarning:

pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy



'오라클 닫힘'

In [5]:
# 1번이라도 스틸을 한 데이터프레임
condition = (my_df['EPICMONSTERSTEALS'] != 0)
steal_df = my_df[condition]
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
steal_df[:30]

# WIN 컬럼을 1 or 0 으로 바꾸기 
steal_df['WIN'] = steal_df['WIN'].apply(lambda x: 1 if x == 'True' else 0)

# 챔피언별 에픽몬스터스틸 성공횟수와 승리횟수
win_df = steal_df.groupby(['CHAMPIONNAME'])[['EPICMONSTERSTEALS','WIN']].sum()
win_df.sort_values(by=['EPICMONSTERSTEALS'],ascending=False)[:10]

# 에픽몬스터스틸을 성공한 챔피언별 총게임판수 
cnt_df = steal_df.groupby(['CHAMPIONNAME'])[['WIN']].count().rename(columns = {'WIN':'CNT'})
cnt_df.sort_values(by=['CNT'],ascending=False)[:10]

# 전체게임에서 등장한 횟수
ap_df = my_df.groupby(['CHAMPIONNAME'])[['WIN']].count().rename(columns = {'WIN':'APCNT'})

# 데이터프레임 합치기 및 스틸 성공시 승률컬럼 추가 
cnt_win_df = pd.merge(win_df,cnt_df,on = ['CHAMPIONNAME'])
cnt_win_df['WINRATE'] = round(cnt_win_df.WIN / cnt_win_df.CNT,4)*100
cnt_win_df.reset_index()
cnt_win_df.sort_values(by=['EPICMONSTERSTEALS'],ascending=False)[:10]

# 데이터프레임 합치기 및 스틸확률컬럼 추가 
result_df = pd.merge(cnt_win_df,ap_df,on = ['CHAMPIONNAME'])
result_df['STEALRATE'] = round(result_df.EPICMONSTERSTEALS/result_df.APCNT,4)*100
result_df.sort_values(by=['STEALRATE'],ascending=False)[:10]

#등장 횟수와 스틸확률 
fig = px.scatter(result_df,
                x = 'APCNT',
                y = 'STEALRATE',
                color = cnt_win_df.index,
                title = '전체게임 등장 횟수와 에픽몬스터 스틸확률'
                )
plotly.offline.iplot(fig)

C:\Users\user\AppData\Local\Temp\ipykernel_4488\595476455.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

